In [57]:
%load_ext autoreload
%autoreload 2

In [58]:
from analyzer import *
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [147]:
labels = ["l1", "l2"]
X, y = make_classification(n_samples=1000, n_features=4, n_informative=2, n_redundant=0, random_state=0, shuffle=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [207]:
analyzer = Analyzer(root_dir="..")

In [228]:
analyzer.model_overview(y_test, y_pred, labels, "1.0", plot_metrics=False, overwrite=True)

{
    "MODEL_NAME": "1.0",
    "ROOT_DIR": "../..",
    "STABLE_ROOT_DIR": "/home/sixtenbohman/dev/python/AI_analyzer",
    "_Analyzer__check_validility": "<function Analyzer.__check_validility at 0x7e8a0057c5e0>",
    "_Analyzer__dump_json": "<function Analyzer.__dump_json at 0x7e8a0057c720>",
    "_Analyzer__get_current_data": "<function Analyzer.__get_current_data at 0x7e8a0057c9a0>",
    "_Analyzer__write_confusion_matrix": "<function Analyzer.__write_confusion_matrix at 0x7e8a0057c4a0>",
    "_Analyzer__write_scores": "<function Analyzer.__write_scores at 0x7e8a0057c860>",
    "__class__": "<attribute '__class__' of 'object' objects>",
    "__delattr__": "<slot wrapper '__delattr__' of 'object' objects>",
    "__dict__": "<attribute '__dict__' of 'Analyzer' objects>",
    "__dir__": "<method '__dir__' of 'object' objects>",
    "__doc__": null,
    "__eq__": "<slot wrapper '__eq__' of 'object' objects>",
    "__format__": "<method '__format__' of 'object' objects>",
    "__ge__": 

AttributeError: 'Analyzer' object has no attribute '__get_current_data'

In [78]:
analyzer.compare_models("1.0", "1.1", overwrite=True)

# Experementing Below

In [183]:
import inspect

class MyClass:
    # Class attributes
    attribute1 = "value1"
    attribute2 = "value2"

    # Class methods
    def method1(self):
        pass

    def _private_method(self):
        pass

    def get_methods(self):
        print({name: value for name, value in inspect.getmembers(self) if not name.startswith('__')})

m = MyClass();
m.get_methods()

{'_private_method': <bound method MyClass._private_method of <__main__.MyClass object at 0x7e8a01996ed0>>, 'attribute1': 'value1', 'attribute2': 'value2', 'get_methods': <bound method MyClass.get_methods of <__main__.MyClass object at 0x7e8a01996ed0>>, 'method1': <bound method MyClass.method1 of <__main__.MyClass object at 0x7e8a01996ed0>>}


In [221]:
string = "this is h"
print(string[:2])
print(string[2:])

th
